# Formation South Green 2021  

##  Initiation à l’analyse de données Minion

### PART 1

Created by J.Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-CIRAD)

Septembre 2021


## 1. Creating the folder, downloading data and so on

### 1.1 Simulated clones

Before starting, please download special data created for this practical training. Data are available on the  from I-Trop server.

Each participant will analyse a Clone, results will be complete in this shared [file](https://lite.framacalc.org/9pd3-ont_sg_2021)

To generate Clone data, a **1Mb** contig was extracted from chromosome 1 of rice.

20 levels of variation were generated and long reads were simulated for each.

We have introduced different variations (SNP, indel, indel+translocations) and also some contaminations. 

In [ ]:
CLONE=Clone10

In [ ]:
cd ~
mkdir -p SG-ONT-2021/DATA
cd SG-ONT-2021/DATA
# download your compressed CloneX 
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/${CLONE}.tar.gz

In [ ]:
#decompress file
tar zxvf ${CLONE}.tar.gz

In [ ]:
# check data 
ls -l ${CLONE}

### 1.2 Real data

Some steps in this practical training can not work in Clones dataset. A second dataset will be download.

These data were generated in this [paper.](https://www.biorxiv.org/content/10.1101/2021.07.04.451066v1.full)

Please decompress Hh dataset.


In [ ]:
cd ~/SG-ONT-2021/DATA
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/real_Hh.tar.gz

In [ ]:
#decompress file
tar zxvf real_Hh.tar.gz

## 2. Basecalling 

When DNA molecules are sequenced, electrical signals are stocked on fast5 format files.

These signals need to be converted on standard fastq files to post-analysis.

Several training dataset models are usually used to convert fast5 to fastq. 


### 2.1 Basecalling with Guppy <span style="color:red"> (Don't run it! ) </span>

Guppy is a data processing toolkit that contains the 'Oxford Nanopore Technologies' basecalling algorithms, and several bioinformatic post-processing features.

Basecalling with guppy can be launch using guppy tool. 

Guppy takes fast5 raw read files and transform electrical signal in fastq files.

We recommend to basecaller yours dataset using a GPU graphic card to obtain results quickly.

Documentation about how to run Guppy on I-trop GPU can be found on https://bioinfo.ird.fr/index.php/tutorials-fr/gpu/


In [ ]:
guppy_basecaller -c dna_r9.4.1_450bps_hac.cfg -i ${INPUT} -r -s ${OUTPUT} --num_callers 4 --gpu_runners_per_device 8 --qscore_filtering --min_qscore 7 -x cuda:${CUDA}

## 3. Quality Control on Long Reads


Calculating data quantity

In [ ]:
cd ~/SG-ONT-2021/DATA
pwd

Calculating how many bases were sequenced

In [ ]:
seqtk seq -A ${CLONE}/ONT/${CLONE}.fastq.gz | grep -v ">" |wc -m

What is the sequencing depth?

### 3.1 Quality Control of FASTQ with Nanoplot (one clone --summary)

Control reads quality using Nanoplot. You can parameter this tool using --help.

In [ ]:
conda activate nanocomp
NanoPlot --help

Launch NanoPlot by your Clone. You can launch NanoPlot using summaries or fastq files.

In [ ]:
# create a folder to save results
mkdir -p ~/SG-ONT-2021/RESULTS
cd ~/SG-ONT-2021/RESULTS

In [ ]:
# run nanoplot 
time NanoPlot -t 1 --summary ../DATA/${CLONE}/ONT/${CLONE}_DeepSimu_sequencing_summary.txt --outdir NANOPLOT_${CLONE}

* NOTE: Clones are simulated data. Check stats on created NanoStats file. <span style="color:red"> it's normal ! it doesn't work! ) </span>

Check stats on created NanoStats file.

In [ ]:
cat NANOPLOT_${CLONE}/NanoStats.txt

* What do you think about data? 

* Estimate coverage.

* What about reads quality qscore?

### 3.2 Quality Control of FASTQ with Nanoplot (Hh real data --fastq )

Check quality using NanoPlot over Hh real data fastq. This can take a while.

In [ ]:
cd ~/SG-ONT-2021/RESULTS/
pwd

**The following Nanoplot will run for a few minutes**

In [ ]:
time NanoPlot -t 1 --fastq ../DATA/real_Hh/H_M1C132_1.fastq.gz --outdir NANOPLOT_Hh

Observe report.html.

* what about this dataset?

### 3.3 Compare reads QC statistics with NanoComp (Summaries dataset)

Compare long reads sequencing datasets using **NanoComp**.

NanoComp compiles quality information in a useful html report.

You can launch NanoComp using summaries or fastq files.

In [ ]:
NanoComp --help


For the moment, we used only a Clone and simulated data. 

Please download a "summaries" obtained in paspalum real data. 

Download available 'Real_PSummaries' to compare them. 


In [ ]:
cd ~/SG-ONT-2021/DATA
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/real_summaries.tar.gz

In [ ]:
#decompress 
tar -zxvf real_summaries.tar.gz

In [ ]:
cd ~/SG-ONT-2021/DATA/real_summaries/
ls -lh  *txt

Compare this 3 summaries files with NanoComp

In [ ]:
time NanoComp --summary OLD_summary.txt FF_summary.txt BON_summary.txt --outdir ~/SG-ONT-2021/RESULTS/NANOCOMP-realsummaries

In [ ]:
conda deactivate

###  Others complemental tools: 

https://github.com/wdecoster/NanoPlot#companion-scripts

* NanoComp: comparing multiple runs

* NanoStat: statistic summary report of reads or alignments

* NanoFilt: filtering and trimming of reads

* NanoLyse: removing contaminant reads (e.g. lambda control DNA) from fastq

FiltLong : filtering long reads by quality https://github.com/rrwick/Filtlong
